In [1]:
!git clone https://github.com/vgsatorras/egnn

Cloning into 'egnn'...
remote: Enumerating objects: 92, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 92 (delta 10), reused 4 (delta 4), pack-reused 66 (from 1)
Receiving objects: 100% (92/92), 297.25 KiB | 5.04 MiB/s, done.
Resolving deltas: 100% (17/17), done.


In [2]:
%cd /content/egnn/models/egnn_clean

/content/egnn/models/egnn_clean


In [3]:
!python egnn_clean.py

In [4]:
import torch
import egnn_clean as eg

In [5]:
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Device: cuda


In [6]:
def get_edges(n_nodes):
    rows, cols = [], []
    for i in range(n_nodes):
        for j in range(n_nodes):
            if i != j:
                rows.append(i)
                cols.append(j)
    edges = [rows, cols]
    return edges

def get_edges_batch(n_nodes, batch_size):
    edges = get_edges(n_nodes)
    edge_attr = torch.ones(len(edges[0]) * batch_size, 1)
    edges = [torch.LongTensor(edges[0]), torch.LongTensor(edges[1])]
    if batch_size == 1:
        return edges, edge_attr
    elif batch_size > 1:
        rows, cols = [], []
        for i in range(batch_size):
            rows.append(edges[0] + n_nodes * i)
            cols.append(edges[1] + n_nodes * i)
        edges = [torch.cat(rows), torch.cat(cols)]
    return edges, edge_attr

# Convert to edge_index
def get_edge_index_batch(n_nodes, batch_size):
    edges, edge_attr = get_edges_batch(n_nodes, batch_size)
    edge_index = torch.stack(edges, dim=0)  # Stack rows and cols
    return edge_index

# Example usage
#n_nodes = 4
#batch_size = 8
#edges, edge_attr = get_edges_batch(n_nodes, batch_size)
#print(edge_attr.shape)
#edge_index = get_edge_index_batch(n_nodes, batch_size)
#print(edge_index.shape)

In [7]:
!pip install torch-sparse -f https://data.pyg.org/whl/torch-$(python -c "import torch; print(torch.__version__)").html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-$(python -c "import torch; print(torch.__version__)").html
!pip install torch-spline-conv -f https://data.pyg.org/whl/torch-$(python -c "import torch; print(torch.__version__)").html
!pip install torch-scatter -f https://data.pyg.org/whl/torch-$(python -c "import torch; print(torch.__version__)").html
!pip install torch-geometric

Looking in links: https://data.pyg.org/whl/torch-2.5.1+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 24.6 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.5.1+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 20.8 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.5.1+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 991.6/991.6 kB 18.9 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.5.1+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 49.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 24.8 MB/s eta 0:00:00


In [8]:
! git clone https://github.com/state-spaces/mamba

Cloning into 'mamba'...
remote: Enumerating objects: 707, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 707 (delta 10), reused 7 (delta 7), pack-reused 694 (from 2)
Receiving objects: 100% (707/707), 1.55 MiB | 18.50 MiB/s, done.
Resolving deltas: 100% (376/376), done.


In [9]:
!pip install ./mamba

Processing ./mamba
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached ninja-1.11.1.3-py3-none-manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata (5.3 kB)
Using cached ninja-1.11.1.3-py3-none-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (422 kB)
  Created wheel for mamba_ssm: filename=mamba_ssm-2.2.4-cp310-cp310-linux_x86_64.whl size=323653202 sha256=c6edff068928b4ceacc6e820a163908c02294fe01dbcc05e95ed4530a5f81e77
  Stored in directory: /tmp/pip-ephem-wheel-cache-g3wz8h00/wheels/fa/fd/f2/375cfbc979bce4a65dc40a325c0f7b1dc7064b0857ea7ae56b
Successfully built mamba_ssm


In [10]:
! pip install triton

  Using cached triton-3.1.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.3 kB)
Using cached triton-3.1.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (209.5 MB)


In [11]:
import argparse
import os.path as osp
from typing import Any, Dict, Optional
import math
import torch as th

import torch
from torch.nn import (
    BatchNorm1d,
    Embedding,
    Linear,
    ModuleList,
    ReLU,
    Sequential,
)
from torch.optim.lr_scheduler import ReduceLROnPlateau
import gc
import torch_geometric.transforms as T
from torch_geometric.datasets import ZINC
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GINEConv, global_add_pool
import inspect
from typing import Any, Dict, Optional

import torch.nn.functional as F
from torch import Tensor
from torch.nn import Dropout, Linear, Sequential

from torch_geometric.nn.conv import MessagePassing
from torch_geometric.nn.inits import reset
from torch_geometric.nn.resolver import (
    activation_resolver,
    normalization_resolver,
)
from torch_geometric.typing import Adj
from torch_geometric.utils import to_dense_batch

from mamba_ssm import Mamba
from torch_geometric.utils import degree, sort_edge_index
import torch_sparse

In [12]:
import torch

def permute_within_batch(x, batch):
    # Enumerate over unique batch indices
    unique_batches = torch.unique(batch)

    # Initialize list to store permuted indices
    permuted_indices = []

    for batch_index in unique_batches:
        # Extract indices for the current batch
        indices_in_batch = (batch == batch_index).nonzero().squeeze()

        # Permute indices within the current batch
        permuted_indices_in_batch = indices_in_batch[torch.randperm(len(indices_in_batch))]

        # Append permuted indices to the list
        permuted_indices.append(permuted_indices_in_batch)

    # Concatenate permuted indices into a single tensor
    permuted_indices = torch.cat(permuted_indices)

    return permuted_indices

# Example usage
batch = torch.tensor([0, 0, 0, 1, 1, 1, 1])
x = torch.tensor([0, 10, 20, 30, 40, 50, 60])

# Get permuted indices
permuted_indices = permute_within_batch(x, batch)

# Use permuted indices to get the permuted tensor
permuted_x = x[permuted_indices]

print("Original x:", x)
print("Permuted x:", permuted_x)
print("Permuted indices:", permuted_indices)

Original x: tensor([ 0, 10, 20, 30, 40, 50, 60])
Permuted x: tensor([ 0, 20, 10, 30, 60, 50, 40])
Permuted indices: tensor([0, 2, 1, 3, 6, 5, 4])


In [13]:
def timestep_embedding(timesteps, dim, max_period=10000):
      """
    Create sinusoidal timestep embeddings.

    :param timesteps: a 1-D Tensor of N indices, one per batch element.
                      These may be fractional.
    :param dim: the dimension of the output.
    :param max_period: controls the minimum frequency of the embeddings.
    :return: an [N x dim] Tensor of positional embeddings.
    """
      half = dim // 2
      freqs = th.exp(
        -math.log(max_period) * th.arange(start=0, end=half, dtype=th.float32) / half
      ).to(device=timesteps.device)
      args = timesteps[:, None].float() * freqs[None]
      embedding = th.cat([th.cos(args), th.sin(args)], dim=-1)
      if dim % 2:
        embedding = th.cat([embedding, th.zeros_like(embedding[:, :1])], dim=-1)
      return embedding

In [14]:
class GPSConv(torch.nn.Module):

    def __init__(
        self,
        channels: int,
        conv: Optional[MessagePassing],
        heads: int = 1,
        dropout: float = 0.0,
        attn_dropout: float = 0.0,
        act: str = 'relu',
        att_type: str = 'transformer',
        order_by_degree: bool = False,
        shuffle_ind: int = 0,
        d_state: int = 16,
        d_conv: int = 4,
        act_kwargs: Optional[Dict[str, Any]] = None,
        norm: Optional[str] = 'batch_norm',
        norm_kwargs: Optional[Dict[str, Any]] = None,
    ):
        super().__init__()

        self.channels = channels
        self.conv = conv
        self.heads = heads
        self.dropout = dropout
        self.att_type = att_type
        self.shuffle_ind = shuffle_ind
        self.order_by_degree = order_by_degree

        assert (self.order_by_degree==True and self.shuffle_ind==0) or (self.order_by_degree==False), f'order_by_degree={self.order_by_degree} and shuffle_ind={self.shuffle_ind}'
        if self.att_type == 'transformer':
            self.attn = torch.nn.MultiheadAttention(
                channels,
                heads,
                dropout=attn_dropout,
                batch_first=True,
            )
        if self.att_type == 'mamba':
            self.self_attn = Mamba(
                d_model=channels,
                d_state=d_state,
                d_conv=d_conv,
                expand=1
            )

        self.mlp = Sequential(
            Linear(channels, channels * 2),
            activation_resolver(act, **(act_kwargs or {})),
            Dropout(dropout),
            Linear(channels * 2, channels),
            Dropout(dropout),
        )

        norm_kwargs = norm_kwargs or {}
        self.norm1 = normalization_resolver(norm, channels, **norm_kwargs)
        self.norm2 = normalization_resolver(norm, channels, **norm_kwargs)
        self.norm3 = normalization_resolver(norm, channels, **norm_kwargs)

        self.norm_with_batch = False
        if self.norm1 is not None:
            signature = inspect.signature(self.norm1.forward)
            self.norm_with_batch = 'batch' in signature.parameters

    def reset_parameters(self):
        r"""Resets all learnable parameters of the module."""
        if self.conv is not None:
            self.conv.reset_parameters()
        self.attn._reset_parameters()
        reset(self.mlp)
        if self.norm1 is not None:
            self.norm1.reset_parameters()
        if self.norm2 is not None:
            self.norm2.reset_parameters()
        if self.norm3 is not None:
            self.norm3.reset_parameters()

    def forward(
        self,
        h: Tensor,
        x: Tensor,
        edge_index: Adj,
        batch: Optional[torch.Tensor] = None,
        **kwargs,
    ) -> Tensor:
        r"""Runs the forward pass of the module."""
        hs = []
        if self.conv is not None:  # Local MPNN.
            y,x = self.conv(h, x, **kwargs)
            y = F.dropout(h, p=self.dropout, training=self.training)
            y = y + h
            if self.norm1 is not None:
                if self.norm_with_batch:
                    y = self.norm1(y, batch=batch)
                else:
                    y = self.norm1(y)
            hs.append(y)
            #print("shape of hs from MPNN:", h.shape)

        ### Global attention transformer-style model.
        if self.att_type == 'transformer':
            y, mask = to_dense_batch(h, batch)
            y, _ = self.attn(y, y, y, key_padding_mask=~mask, need_weights=False)
            h = h[mask]

        if self.att_type == 'mamba':

            if self.order_by_degree:
              # degree() function: edge_index[0]= source nodes of the edges, x.shape[0]= number of nodes)
                deg = degree(edge_index[0], h.shape[0]).to(torch.long)
                order_tensor = torch.stack([batch, deg], 1).T
                # creates a tensor by stacking the 'batch' tensor(batch_index of each node) and the degree of each node. making it easier to reorder nodes
                _, h = sort_edge_index(order_tensor, edge_attr=h)
                # reorders the feature tensor x based on the node degree and batch information, reordering x as well.

            if self.shuffle_ind == 0:
              # no shuffling occurs, and dense batch repr. is created from x
                y, mask = to_dense_batch(h, batch)
                y = self.self_attn(y)[mask]
                # the result is masked to remove invalid entries (padded entries)
                #
            else:
                mamba_arr = []
                # list to store the results of multiple attention passes over shuffled node features
                for _ in range(self.shuffle_ind):
                  # loops over the number of shuffles(shuffle_ind)
                    y_ind_perm = permute_within_batch(h, batch)
                    # Similar to before, converts the shuffled x into a dense representation for each bat
                    y_i, mask = to_dense_batch(h[y_ind_perm], batch)
                    # Similar to before, converts the shuffled x into a dense representation for each batch.
                    y_i = self.self_attn(y_i)[mask][y_ind_perm]
                    # The shuffled dense batch is passed through the self-attention mechanism, and the result is reordered back to the original node order (h_ind_perm).
                    mamba_arr.append(y_i)
                y = sum(mamba_arr) / self.shuffle_ind
                #print("Shape of h from mamba:", y.shape)

                # Averages the results from all shuffled versions to produce the final node representations.
        ###

        y = F.dropout(y, p=self.dropout, training=self.training)
        y = y + h  # Residual connection.
        if self.norm2 is not None:
            if self.norm_with_batch:
                y = self.norm2(y, batch=batch)
            else:
                y = self.norm2(y)
        hs.append(y)
        #print("shape of hs from attention:", h.shape)

        out = sum(hs)  # Combine local and global outputs.
        #print("Shape of out:", out.shape)

        out = out + self.mlp(out)
        if self.norm3 is not None:
            if self.norm_with_batch:
                out = self.norm3(out, batch=batch)
            else:
                out = self.norm3(out)

        return out

    def __repr__(self) -> str:
        return (f'{self.__class__.__name__}({self.channels}, '
                f'conv={self.conv}, heads={self.heads})')

In [15]:
class GraphModel(torch.nn.Module):
    def __init__(self, channels: int, in_node_nf: int, out_node_nf: int, in_edge_nf: int, hidden_nf: int, pe_dim: int, context_dim: int, time_embed_dim: int, num_layers: int, model_type: str, shuffle_ind: int, d_state: int, d_conv: int, order_by_degree: False):
        super().__init__()

        self.node_emb = Linear(64, channels - pe_dim - time_embed_dim - context_dim)
        self.context_emb = Linear(64, context_dim)
        self.context_dim = context_dim
        self.time_embed_dim = time_embed_dim
        self.pe_lin = Linear(20, pe_dim)    # PE_encoder = linear, DeepSet
        self.pe_norm = BatchNorm1d(20)
        self.edge_emb = Linear(4, channels-time_embed_dim)
        self.model_type = model_type
        self.shuffle_ind = shuffle_ind
        self.order_by_degree = order_by_degree
        self.convs = ModuleList()

        for _ in range(num_layers):
            nn = Sequential(
                Linear(channels, channels),
                ReLU(),
                Linear(channels, channels),
            )
            if self.model_type == 'egnn':
                conv = eg.EGNN(in_node_nf, hidden_nf, out_node_nf, in_edge_nf)

            if self.model_type == 'mamba':
                conv = GPSConv(channels, eg.EGNN(in_node_nf, hidden_nf, out_node_nf, in_edge_nf), heads=4, attn_dropout=0.5,
                               att_type='mamba',
                               shuffle_ind=self.shuffle_ind,
                               order_by_degree=self.order_by_degree,
                               d_state=d_state, d_conv=d_conv)

            if self.model_type == 'transformer':
                conv = GPSConv(channels, eg.EGNN(in_node_nf, hidden_nf, out_node_nf, in_edge_nf), heads=4, attn_dropout=0.5, att_type='transformer')
            self.convs.append(conv)

        self.mlp = Sequential(
            Linear(channels, channels // 2),
            ReLU(),
            Linear(channels // 2, channels // 4),
            ReLU(),
            Linear(channels // 4, 1),
        )

    def forward(self, h, pe, x, t, context, edges, edge_index, edge_attr, batch):

        h_pe = self.pe_norm(pe)        # positional_encoding = RWSE-20, LapPE-8, LapPE-16
        time_emb = timestep_embedding(t, self.time_embed_dim)
        context = self.context_emb(context)
        h = torch.cat((self.node_emb(h.squeeze(-1)), self.pe_lin(h_pe), time_emb.squeeze(1), context), dim=1)

        edge_attr = self.edge_emb(edge_attr.squeeze(-1))
        required_repeats = edge_attr.size(0) // time_emb.size(0)
        remainder = edge_attr.size(0) % time_emb.size(0)
        time_emb_edge = time_emb.repeat_interleave(required_repeats, dim=0)

        if remainder > 0:
          extra_time_emb = time_emb[:remainder]
        time_emb_edge = torch.cat((time_emb_edge, extra_time_emb), dim=0)

        assert time_emb_edge.size(0) == edge_attr.size(0), "Final sizes still do not match!"
        edge_attr = torch.cat((edge_attr, time_emb_edge.squeeze(1)), dim=-1)

        for conv in self.convs:
            if self.model_type == 'egnn':
                h = conv(h, x, edges=edges, edge_attr=edge_attr)
            else:
                h = conv(h,x, edge_index, batch, edges=edges, edge_attr=edge_attr)
        h = global_add_pool(h, batch)    # graph_pool = sum, mean
        return self.mlp(h)

In [16]:
# Define hyperparameters
channels = 64
in_node_nf = channels   # Example input node feature size
out_node_nf = channels # Example output node feature size
in_edge_nf = 64    # Example input edge feature size
hidden_nf = 32   # Number of hidden units in layers
pe_dim = 16
time_embed_dim = 16
context_dim = 16      # Positional encoding dimension
num_layers = 3    # Number of layers in the model
model_type = "mamba"  # Graph Neural Network as the type
shuffle_ind = 1    # Enable index shuffling
d_state = 64       # State dimensionality
d_conv = 64        # Convolutional dimensionality
order_by_degree = False  # Do not order nodes by degree

# Initialize the model
model = GraphModel(
    channels=channels,
    in_node_nf=in_node_nf,
    out_node_nf=out_node_nf,
    in_edge_nf=in_edge_nf,
    hidden_nf=hidden_nf,
    pe_dim=pe_dim,
    context_dim=context_dim,
    time_embed_dim=time_embed_dim,
    num_layers=num_layers,
    model_type=model_type,
    shuffle_ind=shuffle_ind,
    d_state=d_state,
    d_conv=d_conv,
    order_by_degree=order_by_degree,
).to(device)

# Print model summary (if `__repr__` or similar is defined)
print(model)

GraphModel(
  (node_emb): Linear(in_features=64, out_features=16, bias=True)
  (context_emb): Linear(in_features=64, out_features=16, bias=True)
  (pe_lin): Linear(in_features=20, out_features=16, bias=True)
  (pe_norm): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (edge_emb): Linear(in_features=4, out_features=48, bias=True)
  (convs): ModuleList(
    (0-2): 3 x GPSConv(64, conv=EGNN(
      (embedding_in): Linear(in_features=64, out_features=32, bias=True)
      (embedding_out): Linear(in_features=32, out_features=64, bias=True)
      (gcl_0): E_GCL(
        (edge_mlp): Sequential(
          (0): Linear(in_features=129, out_features=32, bias=True)
          (1): SiLU()
          (2): Linear(in_features=32, out_features=32, bias=True)
          (3): SiLU()
        )
        (node_mlp): Sequential(
          (0): Linear(in_features=64, out_features=32, bias=True)
          (1): SiLU()
          (2): Linear(in_features=32, out_features=32, bias=True)


In [ ]:
#batch_size = 8
#n_nodes = 4
#n_feat = 64
#x_dim = 3

# Dummy variables h, x and fully connected edges
#h = torch.ones(batch_size *  n_nodes, n_feat)
#x = torch.ones(batch_size * n_nodes, x_dim)
#pe = torch.rand((batch_size* n_nodes, 20))
#batch = torch.zeros(batch_size * n_nodes, dtype=torch.long)
#print(batch.shape)
#edges, edge_attr = eg.get_edges_batch(n_nodes, batch_size)
#print(h.shape)
#print(x.shape)
#print(pe.shape)
#print(batch.shape)
#print(edge_attr.shape)
#edge_index = get_edge_index_batch(n_nodes, batch_size)
# Initialize EGNN
#egnn = eg.EGNN(in_node_nf=n_feat, hidden_nf=32, out_node_nf=64, in_edge_nf=1)

# Run EGNN
#h, x = egnn(h, x, edges, edge_attr)
#print(h.shape, x.shape)

In [17]:
n_nodes = 31
n_edges = 66

h = torch.rand((n_nodes, 64)).to(device)
print(h)
edge_index = torch.randint(0, n_nodes, (2, n_edges)).to(device)
edge_attr = torch.rand((n_edges, 4)).to(device)
edges = [edge_index[0].to(device), edge_index[1].to(device)]
x = torch.rand((n_nodes, 3)).to(device)
pe = torch.rand((n_nodes, 20)).to(device)
batch = torch.zeros(n_nodes, dtype=torch.long).to(device)
ptr = torch.tensor([0, n_nodes]).to(device)
num_graphs = (ptr.shape[0]-1)
print(num_graphs)
print(h.shape)
print(edge_index.shape)
print(edge_attr.shape)
print(x.shape)
print(pe.shape)
print(batch.shape)
print(edges[0].shape)
print(edges[1].shape)

tensor([[0.3427, 0.2709, 0.5347,  ..., 0.4272, 0.9020, 0.8976],
        [0.2671, 0.5069, 0.6776,  ..., 0.9929, 0.8624, 0.4720],
        [0.1337, 0.4303, 0.8587,  ..., 0.4891, 0.7263, 0.0137],
        ...,
        [0.2664, 0.5034, 0.8762,  ..., 0.0420, 0.6542, 0.4533],
        [0.7293, 0.7885, 0.7095,  ..., 0.7060, 0.1579, 0.3635],
        [0.2505, 0.8198, 0.4655,  ..., 0.1269, 0.4545, 0.5341]],
       device='cuda:0')
1
torch.Size([31, 64])
torch.Size([2, 66])
torch.Size([66, 4])
torch.Size([31, 3])
torch.Size([31, 20])
torch.Size([31])
torch.Size([66])
torch.Size([66])


In [18]:
context = torch.randn_like(h, dtype=torch.float).to(device)
print("Shape of context before:", context.shape)
context = context.view(-1, context.size(-1))
print(f"Context shape after repeating: {context.shape}")

t = torch.randint(0, 1000, (num_graphs,)).float().to(device)
print(t)
t = t.view(-1, 1)
t = t[batch]
print(t.shape)

Shape of context before: torch.Size([31, 64])
Context shape after repeating: torch.Size([31, 64])
tensor([385.], device='cuda:0')
torch.Size([31, 1])


In [20]:
output = model( h, pe, x, t, context, edges, edge_index, edge_attr, batch)
print(output.shape)

torch.Size([1, 1])


In [19]:
deg = degree(edge_index[0], h.shape[0]).to(torch.long)
print(deg)
order_tensor = torch.stack([batch, deg], 1).T
print(order_tensor)
_, h = sort_edge_index(order_tensor, edge_attr=h)
print("h after sorting by degree:", h)

tensor([1, 3, 2, 4, 1, 1, 1, 5, 3, 2, 2, 2, 1, 5, 2, 2, 3, 1, 2, 0, 2, 2, 2, 2,
        3, 2, 2, 1, 5, 2, 0], device='cuda:0')
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0],
        [1, 3, 2, 4, 1, 1, 1, 5, 3, 2, 2, 2, 1, 5, 2, 2, 3, 1, 2, 0, 2, 2, 2, 2,
         3, 2, 2, 1, 5, 2, 0]], device='cuda:0')
h after sorting by degree: tensor([[0.2505, 0.8198, 0.4655,  ..., 0.1269, 0.4545, 0.5341],
        [0.1476, 0.3222, 0.3832,  ..., 0.7252, 0.4107, 0.7518],
        [0.3841, 0.3592, 0.4825,  ..., 0.7634, 0.2679, 0.7684],
        ...,
        [0.5645, 0.4209, 0.2905,  ..., 0.6222, 0.7633, 0.0537],
        [0.7457, 0.7225, 0.9545,  ..., 0.0978, 0.1752, 0.5202],
        [0.2664, 0.5034, 0.8762,  ..., 0.0420, 0.6542, 0.4533]],
       device='cuda:0')


In [20]:
import networkx as nx
import torch
import random
import numpy as np

def prioritize_by_eigenvector_centrality(edge_index, num_nodes):
    """
    Prioritize nodes based on eigenvector centrality.

    Args:
        edge_index (torch.Tensor): Edge index of the graph (shape [2, num_edges]).
        num_nodes (int): Number of nodes in the graph.

    Returns:
        torch.Tensor: Tensor of node indices sorted by eigenvector centrality.
    """
    # Create a NetworkX graph from edge_index
    G = nx.Graph()
    edges = edge_index.T.cpu().numpy()
    G.add_edges_from(edges)

    # Compute eigenvector centrality
    centrality = nx.eigenvector_centrality_numpy(G)

    # Get centrality scores and sort nodes
    centrality_scores = torch.tensor(
        [centrality[i] for i in range(num_nodes)], dtype=torch.float
    )
    #prioritized_nodes = torch.argsort(centrality_scores, descending=True)

    return centrality_scores

In [36]:
import networkx as nx
import torch
import community.community_louvain as community_louvain

def permute_by_clusters(edge_index, num_nodes):
    """
    Permute nodes based on clusters defined by the Louvain algorithm.

    Args:
        edge_index (torch.Tensor): Edge index of the graph (shape [2, num_edges]).
        num_nodes (int): Number of nodes in the graph.

    Returns:
        torch.Tensor: Permuted indices of nodes.
    """
    # Convert to NetworkX graph
    G = nx.Graph()
    edges = edge_index.T.cpu().numpy()
    G.add_edges_from(edges)

    # Perform clustering using the Louvain algorithm
    partition = community_louvain.best_partition(G)
    print("Partition:",partition)

    # Group nodes by clusters
    clusters = {}
    for node, cluster_id in partition.items():
        if cluster_id not in clusters:
            clusters[cluster_id] = []
        clusters[cluster_id].append(node)

    # Shuffle nodes within each cluster and concatenate
    permuted_indices = []
    for cluster_nodes in clusters.values():
        permuted_indices += torch.tensor(cluster_nodes)[torch.randperm(len(cluster_nodes))].tolist()

    return torch.tensor(permuted_indices)

In [21]:
centrality_scores = prioritize_by_eigenvector_centrality(edge_index, h.shape[0]).to(device)
print(centrality_scores)
order_tensor = torch.stack([batch, centrality_scores], 1).T
print(order_tensor)
_, h = sort_edge_index(order_tensor, edge_attr=h)
print("h after sorting by degree:", h)

tensor([0.0125, 0.1368, 0.1180, 0.3434, 0.1748, 0.0848, 0.2841, 0.3206, 0.1192,
        0.2601, 0.1599, 0.1411, 0.0520, 0.2906, 0.0825, 0.1570, 0.1278, 0.0025,
        0.1427, 0.0582, 0.0598, 0.0444, 0.1873, 0.1621, 0.0271, 0.2096, 0.2428,
        0.0258, 0.3830, 0.1017, 0.1171], device='cuda:0')
tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000],
        [0.0125, 0.1368, 0.1180, 0.3434, 0.1748, 0.0848, 0.2841, 0.3206, 0.1192,
         0.2601, 0.1599, 0.1411, 0.0520, 0.2906, 0.0825, 0.1570, 0.1278, 0.0025,
         0.1427, 0.0582, 0.0598, 0.0444, 0.1873, 0.1621, 0.0271, 0.2096, 0.2428,
         0.0258, 0.3830, 0.1017, 0.1171]], device='cuda:0')
h after sorting by degree: tensor([[0.9710, 0.7370, 0.7639,  ..., 0.8878, 0.2152, 0.2150],
        [0.2505, 0.8198

In [29]:
mamba_arr = []
shuffle_ind = 1
for _ in range(shuffle_ind):
  y_ind_perm = permute_within_batch(h, batch)
  y_i, mask = to_dense_batch(h[y_ind_perm], batch)

print("y_ind_perm:", y_ind_perm)
print(h[y_ind_perm])

y_ind_perm: tensor([ 0, 19, 21, 30,  8, 18,  5,  7, 29, 13,  1,  3,  2, 11, 20, 10,  9, 17,
        14, 27, 26, 25, 23, 16, 28,  6, 24, 12, 15, 22,  4], device='cuda:0')
tensor([[0.9710, 0.7370, 0.7639,  ..., 0.8878, 0.2152, 0.2150],
        [0.5406, 0.7842, 0.9144,  ..., 0.6539, 0.4140, 0.1852],
        [0.9816, 0.0697, 0.2098,  ..., 0.3765, 0.9406, 0.0673],
        ...,
        [0.1476, 0.3222, 0.3832,  ..., 0.7252, 0.4107, 0.7518],
        [0.9161, 0.1328, 0.4911,  ..., 0.2706, 0.3738, 0.7482],
        [0.1337, 0.4303, 0.8587,  ..., 0.4891, 0.7263, 0.0137]],
       device='cuda:0')


In [37]:
print(h)
permuted_indices = permute_by_clusters(edge_index, h.shape[0])
assert permuted_indices.ndim == 1, "Permuted indices should be a 1D tensor."
# Use permuted indices to get the permuted tensor
permuted_h = h[permuted_indices]
print("Permuted h:", permuted_h)

tensor([[ 0.8847,  0.9463, -1.1158,  ..., -0.5613, -1.1551, -0.1512],
        [ 0.6538,  0.1757,  1.1467,  ..., -0.7878, -2.2434, -0.2146],
        [ 0.5866,  1.2984,  0.3276,  ..., -0.9381, -0.2760, -0.2766],
        ...,
        [-1.3188, -0.5298,  0.0390,  ...,  1.3245, -0.5754,  0.1074],
        [-1.4053,  0.9556,  0.5469,  ..., -1.7763,  0.3285,  0.9740],
        [ 1.0622, -1.1467, -0.3092,  ...,  0.7172,  0.1658,  0.6781]])
Partition: {0: 0, 1: 0, 2: 3, 3: 3, 4: 1, 5: 1, 6: 2, 7: 2}
Permuted h: tensor([[ 0.8847,  0.9463, -1.1158,  ..., -0.5613, -1.1551, -0.1512],
        [ 0.6538,  0.1757,  1.1467,  ..., -0.7878, -2.2434, -0.2146],
        [ 0.5866,  1.2984,  0.3276,  ..., -0.9381, -0.2760, -0.2766],
        ...,
        [-0.0818, -0.7216, -0.7346,  ..., -0.4065, -1.7560, -0.2450],
        [ 1.0622, -1.1467, -0.3092,  ...,  0.7172,  0.1658,  0.6781],
        [-1.4053,  0.9556,  0.5469,  ..., -1.7763,  0.3285,  0.9740]])
